<a href="https://colab.research.google.com/github/prhld0111/Mini-Project-3/blob/main/Solution1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 10
num_classes = 7  # FER-2013 has 7 emotion classes

# Data Transformations
data_transforms = {
    "train": transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
        transforms.Resize((224, 224)),  # ResNet50 input size
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
    ]),
    "val": transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

# Load Dataset
data_dir = "path_to_fer2013"  # Replace with your dataset path
image_datasets = {
    "train": datasets.ImageFolder(root=f"{data_dir}/train", transform=data_transforms["train"]),
    "val": datasets.ImageFolder(root=f"{data_dir}/val", transform=data_transforms["val"]),
}

dataloaders = {
    "train": DataLoader(image_datasets["train"], batch_size=batch_size, shuffle=True),
    "val": DataLoader(image_datasets["val"], batch_size=batch_size, shuffle=False),
}

# Load Pretrained Model
model = models.resnet50(pretrained=True)

# Modify the Final Layer for Emotion Classes
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    history = {"train_loss": [], "val_loss": []}

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluation mode

            running_loss = 0.0
            correct_predictions = 0
            total_samples = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # Backward + optimize only in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Track statistics
                running_loss += loss.item() * inputs.size(0)
                correct_predictions += torch.sum(preds == labels.data)
                total_samples += labels.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct_predictions.double() / total_samples

            history[f"{phase}_loss"].append(epoch_loss)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model, history

# Train and Evaluate the Model
model, history = train_model(model, dataloaders, criterion, optimizer, num_epochs)

# Plot Training and Validation Loss
plt.plot(history["train_loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Save the Model
torch.save(model.state_dict(), "emotion_recognition_resnet50.pth")
print("Model saved!")